# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Load csv that was exported to DF
colnames = ['City', 'Latitude', 'Longitude', 'Max_Temp', 'Humidity', 'Cloudy', 'Wind', 'Country', 'Date']
data = pd.read_csv('City_Weather.csv', names=colnames)

data_df = data.drop([data.index[0]])

# Drop NaN
data_df.dropna(inplace = True) 



### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = data_df[["Latitude", "Longitude"]].astype(float)

# Add Humid as Weights
humid = data_df["Humidity"].astype(float)

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig                 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
data_df.head()

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudy,Wind,Country,Date
1,rikitea,-23.12,-134.97,75.67,85.0,100.0,15.82,PF,1592519182.0
2,banikoara,11.3,2.44,79.93,69.0,85.0,6.6,BJ,1592519080.0
3,vaini,-21.2,-175.2,75.2,78.0,20.0,12.75,TO,1592519155.0
4,busselton,-33.65,115.33,51.71,76.0,46.0,3.27,AU,1592519227.0
5,pisco,-13.7,-76.22,66.2,77.0,75.0,13.87,PE,1592518751.0


In [6]:
# Set filters on ideal vacation city

# convert max temp and clouds to numbers
data_df['Max_Temp'] = pd.to_numeric(data_df['Max_Temp'])
data_df['Cloudy'] = pd.to_numeric(data_df['Cloudy'])
data_df['Humidity'] = pd.to_numeric(data_df['Humidity'])

# temp between 75-90
mask = (data_df['Max_Temp'] > 75) & (data_df['Max_Temp'] < 90)
vac_city1 = data_df.loc[mask]

#Clouds less than 20
vac_city2 = vac_city1.loc[(vac_city1['Cloudy'] < 20)]

#humidity less than 60
vac_city3 = vac_city2.loc[(vac_city2['Humidity'] < 25)]
vac_city3

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudy,Wind,Country,Date
75,saint george,37.1,-113.58,89.60,9.0,1.0,10.29,US,1592518627.0
91,gazojak,41.19,61.4,75.04,18.0,6.0,4.79,TM,1592519238.0
148,matagami,49.75,-77.63,89.60,24.0,1.0,12.75,CA,1592519055.0
178,srednekolymsk,67.45,153.68,84.52,19.0,18.0,5.48,RU,1592519248.0
188,balkanabat,39.51,54.37,86.32,13.0,0.0,17.43,TM,1592519054.0
207,sabha,27.04,14.43,77.83,18.0,0.0,2.39,LY,1592519180.0
257,grants,35.15,-107.85,86.00,6.0,1.0,11.41,US,1592519002.0
303,awjilah,29.11,21.29,79.79,24.0,0.0,8.66,LY,1592519068.0
312,agadez,19.75,10.25,87.21,15.0,16.0,4.7,NE,1592519264.0
463,aswan,24.09,32.91,86.00,17.0,14.0,20.8,EG,1592519096.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# Set dataframe to hotel_df
hotel_df = vac_city3

# Store 'Lat' and 'Lng' into  locations 
locations = hotel_df[["Latitude", "Longitude"]].astype(float)
#print(locations)
num_loc = len(locations)
num_loc

14

In [7]:

# geocoordinates
#######################################################################
#  Loop through lat and lng
target_coordinates = "37.1, -113.58"


target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the name and address of the first restaurant that appears
print(places_data["results"][0]["name"])


#Add column to dataframe for hotel name
#hotel_df['Hotel Name'] = 



Best Western Plus Abbey Inn


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
